In [1]:
# Import Dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Data Source: BulbaPedia
url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number'

In [3]:
# Create soup response to find tables
response=requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')

In [4]:
pkmn_tables=soup.find_all('table')[1:8]

In [5]:
# Add Alolan forms to first stat table (Kanto)
kanto_df=pd.read_html(pkmn_tables[0].prettify())[0]
kanto_df[3][kanto_df[0].isnull()]='Alolan '+kanto_df[3][kanto_df[0].isnull()]

In [6]:
# merge pkmn tables from all generations
all_pkmn_df=kanto_df[1:]
for gen in range(1,7):
#     trim header row to enable merge
    gen_df = pd.read_html(pkmn_tables[gen].prettify())[0][1:]
    all_pkmn_df=all_pkmn_df.append(gen_df)
# reindex combined table
count=len(all_pkmn_df)
indices=[i for i in range(count)]
all_pkmn_df.index=indices

In [7]:
# Drop regional pokedex number and picture column
nat_pkmn_df=all_pkmn_df[[1,3,4,5]]
# Convert national pokedex numbers to integers
count=len(nat_pkmn_df)
for form in range(count):
    nat_pkmn_df[1][form]=int(nat_pkmn_df[1][form][1:])
# Rename Columns
nat_pkmn_df=nat_pkmn_df.rename(columns={1:'nat_dex',3:'name',4:'type1',5:'type2'})
# Add forme names to Giratina and Thundurus
nat_pkmn_df.iloc[521]['name']='Giratina (Altered Forme)'
nat_pkmn_df.iloc[522]['name']='Giratina (Origin Forme)'
nat_pkmn_df.iloc[683]['name']='Thundurus (Incarnate Forme)'
# insert Thundurus Therian Forme
nat_pkmn_df[nat_pkmn_df.index > 683].index += 1
nat_pkmn_df=nat_pkmn_df.append({'nat_dex':642,'name':'Thundurus (Therian Forme)','type1':'Electric','type2':'Flying'},ignore_index=True)
nat_pkmn_df=nat_pkmn_df.rename(index={856:684})
nat_pkmn_df=nat_pkmn_df.sort_index()

nat_pkmn_df

C:\Users\OW93790\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


nat_dex             name     type1     type2
0         1        Bulbasaur     Grass    Poison
1         2          Ivysaur     Grass    Poison
2         3         Venusaur     Grass    Poison
3         4       Charmander      Fire       NaN
4         5       Charmeleon      Fire       NaN
5         6        Charizard      Fire    Flying
6         7         Squirtle     Water       NaN
7         8        Wartortle     Water       NaN
8         9        Blastoise     Water       NaN
9        10         Caterpie       Bug       NaN
10       11          Metapod       Bug       NaN
11       12       Butterfree       Bug    Flying
12       13           Weedle       Bug    Poison
13       14           Kakuna       Bug    Poison
14       15         Beedrill       Bug    Poison
15       16           Pidgey    Normal    Flying
16       17        Pidgeotto    Normal    Flying
17       18          Pidgeot    Normal    Flying
18       19          Rattata    Normal       NaN
19       19   Alolan Rattata      Dark    Normal
20       20         Raticate    Normal       NaN
21       20  Alolan Raticate      Dark    Normal
22       21          Spearow    Normal    Flying
23       22           Fearow    Normal    Flying
24       23            Ekans    Poison       NaN
25       24            Arbok    Poison       NaN
26       25          Pikachu  Electric       NaN
27       26           Raichu  Electric       NaN
28       26    Alolan Raichu  Electric   Psychic
29       27        Sandshrew    Ground       NaN
..      ...              ...       ...       ...
826     780           Drampa    Normal    Dragon
827     781         Dhelmise     Ghost     Grass
828     782         Jangmo-o    Dragon       NaN
829     783         Hakamo-o    Dragon  Fighting
830     784          Kommo-o    Dragon  Fighting
831     785        Tapu Koko  Electric     Fairy
832     786        Tapu Lele   Psychic     Fairy
833     787        Tapu Bulu     Grass     Fairy
834     788        Tapu Fini     Water     Fairy
835     789           Cosmog   Psychic       NaN
836     790          Cosmoem   Psychic       NaN
837     791         Solgaleo   Psychic     Steel
838     792           Lunala   Psychic     Ghost
839     793         Nihilego      Rock    Poison
840     794         Buzzwole       Bug  Fighting
841     795        Pheromosa       Bug  Fighting
842     796        Xurkitree  Electric       NaN
843     797       Celesteela     Steel    Flying
844     798          Kartana     Grass     Steel
845     799         Guzzlord      Dark    Dragon
846     800         Necrozma   Psychic       NaN
847     801         Magearna     Steel     Fairy
848     802        Marshadow  Fighting     Ghost
849     803          Poipole    Poison       NaN
850     804        Naganadel    Poison    Dragon
851     805        Stakataka      Rock     Steel
852     806      Blacephalon      Fire     Ghost
853     807          Zeraora  Electric       NaN
854     808           Meltan     Steel       NaN
855     809         Melmetal     Steel       NaN

[857 rows x 4 columns]

In [8]:
# Save data to csv
nat_pkmn_df.to_csv('Output/nat_dex.csv')